<a href="https://colab.research.google.com/github/mbCap/Fallstudie_BESS_Degradation/blob/main/Latest_von_Fallstudie_BESS_Degradation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fallstudie BESS

In [4]:
!git clone https://github.com/AlexKressner/WS24_Supply_Chain_Optimierung

fatal: destination path 'WS24_Supply_Chain_Optimierung' already exists and is not an empty directory.


In [5]:
!pip install -q pyscipopt latest

In [6]:
import pandas as pd  # Import der Pandas-Bibliothek zur Datenverarbeitung
from pyscipopt import Model, quicksum  # Import der SCIP-Optimierungsbibliothek

# Ordnerpfad für die Daten
folder = "WS24_Supply_Chain_Optimierung/Daten/Fallstudie"

# Laden der Strompreisprognosen aus einer Excel-Datei
preisprognose = pd.read_excel(f"{folder}/Preisprognosen.xlsx")

# Definiere den Zeitbereich für 24 Stunden
T = range(1, 25)

# Parameter des Batteriesystems
cap = 40.0  # Nominale Kapazität in Megawattstunden (MWh)
DoD = 0.80  # Depth of Discharge (maximaler Entladungsgrad)
SOC_min = cap * (1 - DoD)  # Minimaler Ladezustand in MWh
SOC_max = cap  # Maximaler Ladezustand in MWh
c_rate = 0.5  # Maximale Lade-/Entladeleistung als Anteil der Kapazität
eta_rte = 0.975  # Round-trip-Effizienz (Verluste beim Laden und Entladen)
eta_wr = 0.985  # Wechselrichter-Wirkungsgrad
cyclecost = 1500.0  # Fixkosten pro Zyklusdurchlauf in Euro
maxCycles = 2.0  # Maximale Anzahl von Lade-/Entladezyklen pro Tag

# Berechnung der erwarteten Marktpreise je Stunde
p_Markt = preisprognose.groupby("Stunde")["Strompreis"].mean().tolist()

# Effizienzverluste beim Laden und Entladen
eta_charge = eta_rte * eta_wr  # Gesamteffizienz beim Laden
eta_discharge = eta_rte * eta_wr  # Gesamteffizienz beim Entladen

# Funktion zur Berechnung der degradierten Batteriekapazität
def get_degraded_capacity(years):
    return cap * (1 - 0.02) ** years  # Annahme: 2% Kapazitätsverlust pro Jahr

# Funktion zur Initialisierung des Optimierungsmodells
def initialize_model(degraded_capacity):
    scip = Model()  # SCIP-Optimierungsmodell erstellen

    # Entscheidungsvariablen für den Ladezustand der Batterie
    SOC = {t: scip.addVar(vtype="CONTINUOUS", lb=SOC_min, ub=SOC_max, name=f"SOC_{t}") for t in T}

    # Entscheidungsvariablen für Laden und Entladen der Batterie
    charge = {t: scip.addVar(vtype="CONTINUOUS", lb=0, ub=c_rate * degraded_capacity, name=f"charge_{t}") for t in T}
    discharge = {t: scip.addVar(vtype="CONTINUOUS", lb=0, ub=c_rate * degraded_capacity, name=f"discharge_{t}") for t in T}

    # Variable für die Anzahl der genutzten Lade-/Entladezyklen
    Z = scip.addVar(vtype="CONTINUOUS", lb=0, ub=maxCycles, name="cycles_used")

    # Anfangs- und Endbedingungen für den Ladezustand
    scip.addCons(SOC[1] == 0.5 * degraded_capacity)  # Start bei 50% Kapazität
    scip.addCons(SOC[24] == 0.5 * degraded_capacity)  # Ende bei 50% Kapazität
    scip.addCons(discharge[24] == 0)  # Keine Entladung in der letzten Stunde

    # Ladezustands-Dynamik für jede Stunde
    for t in T:
        if t == 1:
            continue
        scip.addCons(SOC[t] == SOC[t - 1] + eta_charge * charge[t - 1] - (1 / eta_discharge) * discharge[t - 1])

    # Begrenzung der Zyklenzahl basierend auf Durchsatz
    throughput = quicksum(charge[t] + discharge[t] for t in T)
    scip.addCons(Z >= throughput / (2.0 * degraded_capacity * DoD))
    scip.addCons(Z <= maxCycles)

    # Zielfunktion: Maximiere den Gewinn aus dem Handel am Strommarkt
    scip.setObjective(quicksum(p_Markt[t - 1] * discharge[t] - p_Markt[t - 1] * charge[t] for t in T) - cyclecost * Z, sense="maximize")

    return scip

# Funktion zur Berechnung des täglichen und jährlichen Erlöses
def calculate_revenue_for_year_and_day(years, scip):
    degraded_capacity = get_degraded_capacity(years)  # Berechnung der reduzierten Kapazität
    scip = initialize_model(degraded_capacity)  # Initialisierung des Modells

    scip.optimize()  # Optimierung ausführen

    daily_revenue = scip.getObjVal()  # Optimierter Tageserlös
    annual_revenue = daily_revenue * 365  # Hochrechnung auf das ganze Jahr

    return daily_revenue, annual_revenue

# Ausgabe des Erlöses für jedes Jahr unter Berücksichtigung der Degradation
print("Erlös pro Jahr und pro Tag unter Berücksichtigung der Degradation:")

# Berechnung für Jahr 0 (keine Degradation)
revenue_today, annual_revenue_today = calculate_revenue_for_year_and_day(0, None)
print(f"Jahr 0: {revenue_today:.2f} € pro Tag / {annual_revenue_today:.2f} € pro Jahr")

# Berechnung für die nächsten 10 Jahre unter Annahme von 2% jährlicher Kapazitätsdegradation
for years in range(1, 11):
    revenue_in_day, revenue_in_year = calculate_revenue_for_year_and_day(years, None)
    revenue_loss_per_day = revenue_today - revenue_in_day
    revenue_loss_per_year = annual_revenue_today - revenue_in_year
    print(f"Jahr {years}: {revenue_in_day:.2f} € pro Tag / {revenue_in_year:.2f} € pro Jahr "
          f"(Erlösverlust pro Tag: {revenue_loss_per_day:.2f} € / Erlösverlust pro Jahr: {revenue_loss_per_year:.2f} €)")

# Berechnung der prozentualen Veränderung des Tageserlöses nach 10 Jahren
revenue_loss_percentage = ((revenue_today - revenue_in_day) / revenue_today) * 100
print("\nWie verändert sich der Erlös mit Batterie-Degradation?")
print(f"Der Tageserlös sinkt im 10. Jahr um {revenue_loss_percentage:.2f}% im Vergleich zum Ausgangsjahr.")


Erlös pro Jahr und pro Tag unter Berücksichtigung der Degradation:
Jahr 0: 1885.12 € pro Tag / 688068.14 € pro Jahr
Jahr 1: 1873.13 € pro Tag / 683691.29 € pro Jahr (Erlösverlust pro Tag: 11.99 € / Erlösverlust pro Jahr: 4376.85 €)
Jahr 2: 1845.18 € pro Tag / 673490.04 € pro Jahr (Erlösverlust pro Tag: 39.94 € / Erlösverlust pro Jahr: 14578.10 €)
Jahr 3: 1816.36 € pro Tag / 662972.94 € pro Jahr (Erlösverlust pro Tag: 68.75 € / Erlösverlust pro Jahr: 25095.20 €)
Jahr 4: 1786.84 € pro Tag / 652195.68 € pro Jahr (Erlösverlust pro Tag: 98.28 € / Erlösverlust pro Jahr: 35872.46 €)
Jahr 5: 1756.59 € pro Tag / 641153.86 € pro Jahr (Erlösverlust pro Tag: 128.53 € / Erlösverlust pro Jahr: 46914.28 €)
Jahr 6: 1725.60 € pro Tag / 629842.99 € pro Jahr (Erlösverlust pro Tag: 159.52 € / Erlösverlust pro Jahr: 58225.16 €)
Jahr 7: 1693.86 € pro Tag / 618258.43 € pro Jahr (Erlösverlust pro Tag: 191.26 € / Erlösverlust pro Jahr: 69809.72 €)
Jahr 8: 1661.36 € pro Tag / 606395.46 € pro Jahr (Erlösverlust 

<ipython-input-6-77ab8b52adb0>:37: UserWarning: linked SCIP 9.02 is not recommended for this version of PySCIPOpt - use version 9.2.1
  scip = Model()  # SCIP-Optimierungsmodell erstellen
